## Calculate a chemical dissimilarity matrix using the chemical structural compositional similarity metric

**Author**: Jeremy Owen (Jeremy.Owen@vuw.ac.nz) <br>
**Edited by**: Madeleine Ernst (mernst@ucsd.edu) <br>
**Use case**: Calculate a chemical dissimilarity matrix using the chemical structural compositional similarity metric (CSCS) proposed by and adapted from (Sedio et al. 2017, Sources of variation in foliar secondary chemistry in a tropical forest tree community, Ecology 98, 616-623). This metric compares MS1 feature intensities in between samples and takes into account the feature's strucutral similarity by multiplying with the cosine score of resepctive MS2 spectra.<br>
**Input file format**: <br>
<ul>
<li>**Edges table** (.tsv) retrieved from GNPS output (networkedges_selfloop folder). </li>
<li>**Feature table** (.csv) with features in rows and samples in columns. This corresponds to the MZmine output table, selecting only features with associated MS2 data. </li> 
<li>**Attribute table** (.tsv) file with 2 columns, the first column specifying any character string you want to add to the sample name, and the second column containing your sample names. </li>
</ul>
**Outputs**: CSCS distance matrix, which can be used as input to create a PCoA plot in Emperor (using CSCS_PCoA_Emperor.ipynb) and dendrogram as .pdf output. <br>
**Dependencies**: Python 3, numpy, scipy, matplotlib, pandas

In [32]:
"""
Created on Tue Feb 28 22:38:27 2017

@author: SpiffKringle
"""
import numpy as np
import pickle
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import squareform
import matplotlib.pyplot as plt
import sys
import pandas as pd

Specify the name of your attribute table

In [33]:
genus_file = "Alnus_Metadata.txt"

Specify a method for calculating the distance for hierarchical cluster analysis (dendrogram output). <br>
You can choose between *single*, *average*, *complete*, *weighted* and *centroid*

In [34]:
linkage_type = "complete"

Specify if MS1 intensities should be treated as binary data

In [35]:
binary = False

In [50]:
SubDict = {}

with open(genus_file, 'r') as F:
    for line in F:
        name = line.split()[1]
        SubDict[name] = name
        #subgenus =  line.split()[1]+ "_" + line.split()[0].split("_")[1]
        #SubDict[name] = subgenus

In [37]:
SubDict

{'AF-T': 'AF-T',
 'AF-L': 'AF-L',
 'AF-B': 'AF-B',
 'AF-F': 'AF-F',
 'Aj-T': 'Aj-T',
 'Aj-L': 'Aj-L',
 'Aj-B': 'Aj-B',
 'Aj-F': 'Aj-F',
 'Ah-T': 'Ah-T',
 'Ah-L': 'Ah-L',
 'Ah-F': 'Ah-F',
 'Ahv-T': 'Ahv-T',
 'Ahv-L': 'Ahv-L',
 'Ahv-F': 'Ahv-F',
 'Ahv-B': 'Ahv-B'}

In [38]:
def ReadNodesFile(NodesFile,remove_start = 1, cutoff = 10,
                  blank_name=None, sep = ",",binary = binary):
    
    """
    * reads a nodes.out file from GNPS and returns: 
        > NameDict - a dictionary of form{columnName:index} 
        > VectorDict - a dictionary of the form {nodeID:[list of intensities for each sample}
    * at this stage intensities are actualy number of scans (roughly proportional)
    * Assumes that blanks are labelled "blank"
    * in cases where sample values/blank values < cutoff, sample values are set to zero
    * This removes "contaminant" ions that were seen in the blank sample to a comprably high level
    """
    
    print("mod1")
    with open(NodesFile, 'r') as F:
        #names = F.readline().split(sep)[remove_start:-remove_end]
        names = F.readline().split(sep)[remove_start:]
        names[-1] = names[-1].strip()
        print("analysing the following samples:")
        print("***")
        
        for name in names:
            print(name)
        print("***")
        NameDict = {name:names.index(name) for name in names}
        
        if blank_name:
            blank = NameDict[blank_name]
        
        else:
            blank = None
        VectorDict = {}
        
        for line in F:
            data = line.split(sep)
            NodeID = data[0]
            vector = data[remove_start:]
            
            if binary:
                CleanVector = [1.0 if float(i) > 1000 else 0 for i in vector ]
            
            else:
                CleanVector = [float(i) for i in vector]
            VectorDict[NodeID] = CleanVector
    
    return (NameDict,VectorDict)
    # the output of ReadNodesFile is a tuple, consisting of 2 elements, the first element RnF[0], consists
    # of a dictionary with length 43, each with the sample name and associated sample number
    # RnF[1] consists of 5652 dictionaries (one per ion), each of these
    # dictionaries consists of 43 items (one entry per sample)
           
 # test ReadEdgesFile:
    #ReF = ReadEdgesFile("/Users/madeleineernst/Documents/PostDoc/Jeremy/edges.txt")   

In [39]:
def ReadEdgesFile(EdgesFile, cutoff = 0.65):
    """
    Parses the edge file from GNPS (cosine score)
    """
    with open(EdgesFile, 'r') as F:
        PairSet = set({})
        Weights = {}
        headers = F.readline().split("\t")
        #return headers
        for line in F:
            data = line.split("\t")
            i = data[0]
            j = data[1]
            cosine = float(data[4])
            if cosine > cutoff:
                Weights[(i,j)] = cosine
                PairSet.add((i,j))
                PairSet.add((j,i))
    return (Weights,PairSet)
    # the output of ReadEdgesFile consists of a tuple of length 2, ReF[0] is a dictionary of length 7188 (lenght of edge file), 
    # listing cosine scores of all node connections (names) 
    # ReF[1] is a set with length 11463, consisting of all possible network node partners?

In [40]:
def MakeSparseMatrix(NameDict,VectorDict):
    
    """
    ############
    What it does
    ############
    
    Intially Transposes the output of ReadNodes to give a dictionary, SampleVectors
    which has the form:
        {SampleName:np.array([ion intensities...])}
        Where all ion intensities are normailized (summ to 1)
        
    Then removes all of the zero data to give a sparse (nested dictionary) representation as follows:
        {SampleName:{Node1:intensity1, Node2:intensity2, Node5:intensity5 ...}}
        In the above representation, absense of Node:intensity data indicates the intensity of the
        node in question was zero
    
    ###############
    what it returns
    ###############
    
    SampleVectors - A sparse representation of nodes present in each sample
    NodeIDs - A list of IDs for all nodes in the sample
    NodeDict - Dictionary of form {NodeID:index in NodeIDs, not sure that this is used for anything}
    """
    
    AllZeros = []
    NodeIDs = list(VectorDict.keys())
    NodeDict = {NodeID:NodeIDs.index(NodeID) for NodeID in NodeIDs}
    SampleVectors = {}
    
    for name in NameDict:
        index = NameDict[name]
        data = [VectorDict[NodeID][index] for NodeID in NodeIDs]#transpose
        SampleVectors[name] = np.array(data)#make it into an array
       
    for sample in SampleVectors:#sum the ion intensities in the array
        TmpVector = np.copy(SampleVectors[sample])
        IonSum = sum(TmpVector)
        
        if IonSum > 0:#divide through by the ion sum to normalize to one
            TmpVector = TmpVector/IonSum
            NewEntry = {}
            
            for i in range(len(TmpVector)):#get rid of the zeros and make a sparse representation of the data
                if TmpVector[i] > 0:
                    NewEntry[NodeIDs[i]] = TmpVector[i]
            SampleVectors[sample] = NewEntry
                         
        else:
            SampleVectors[sample] = None
            AllZeros.append(sample)#track the things which have no data (all node intensities are zeros)
            
    for i in AllZeros:
        SampleVectors.pop(i,None)
        
    return(SampleVectors,NodeIDs,NodeDict)
    # return a tuple with 3 elements
    # MsM[0] has length 43, consisting of 43 dictionaries (one for each sample)

    # ReF[1] corresponds to ValidNodes
    # this function will be called within MakePairScores, skip this when running 
    # functions individually and directly run MakePairScores, which implements
    # this function on every possible species pair

**Calculate MS1 intensities as min(A[i],B[j])/max(A[i],B[j]):** <br>

output[(i,j)] = min(A[i],B[j])/max(A[i],B[j]) <br>#output[(i,j)] = A[i] * B[j] <br>

**or A[i] * B[j]:**   <br>

#output[(i,j)] = min(A[i],B[j])/max(A[i],B[j]) <br>
output[(i,j)] = A[i] * B[j]

In [41]:
def CompareSpectralVectors(A,B,ValidNodes):
    """
    Takes two sparse vectors of normalised node intensities (A,B),
    performs all against all comparison of entries
    and returns a single dictionary (sparse matrix) of intensity similaritres.
    Matrix entries are zero (absent from dict) if either Ai or Bj is zero
    else min/max. e.g:
    
       A [1][2]
      B
     [1]  1 .5
     [0]  0  0
     
    """
    output = {}
    
    for i in A:
        for j in B:
            if (i,j) in ValidNodes:
                output[(i,j)] = min(A[i],B[j])/max(A[i],B[j])
                #output[(i,j)] = A[i] * B[j]
                
    return output
    
# SampleVectors = SampleMatrix[0]
# MpS = MakePairScores(MsM[0],ReF[1])

In [42]:
def MakePairScores(SampleVectors,ValidNodes,limit = None):
    
    """
    Runs all-against-all CompareSpectralVectors for samples (including self-self)
    PairScores looks like this:
        {(A,B):{(1,1):0.7,(1,2):0.3,(2,1):0.01...},A,C:{(1,1)...}}
        A,B,C = sample names
        (1,1),(1,2) ... are tuples of nodeIDs
        the values are <0.0-1.0. Similarity in intenstities
    """
    
    PairScores = {}
    samples = list(SampleVectors.keys())
    
    if limit:
        samples = samples[:limit]
    StartIndex = 0
    
    for i in samples:
        print("...Making comparisons for sample " + str(i))
        for j in samples[StartIndex:]:
            PairScores[(i,j)] = CompareSpectralVectors(SampleVectors[i],SampleVectors[j],ValidNodes)
            
    return PairScores
    # returns a dictionary of length 1849 (one entry for each species pair), each dictionary lists then pairwise
    # entries of ion intensities between these two samples
    ##
    # pairscores contains all pairwise single values of min(A[i],B[j])/max(A[i],B[j])
    ##
                  
    # WbC = WeightByCosine(MpS,ReF[0])

In [43]:
def WeightByCosine(pairs,CosineScores,SelfBonus = 1,OtherPenalty = 1):
    
    """
    Takes the sparse matrix dictionary of sample comparisons output by
    MakePairScores, and wieghts the entries by cosine similarity of the 
    two compounds represented at each position
    """ 
    
    for pair in pairs:
        for nodes in pairs[pair]:
            
            #print(nodes)
            if nodes[0] == nodes[1]:
                pairs[pair][nodes] *= SelfBonus
                     
            elif nodes in CosineScores:
                pairs[pair][nodes] *= CosineScores[nodes]**OtherPenalty
                     
            else:
                pairs[pair][nodes] *= CosineScores[(nodes[1],nodes[0])]**OtherPenalty
# this funciton does not return anything
        
# SP = SumPairs(MpS)

In [44]:
def SumPairs(pairs):
    SummedValues = {}
    for pair in pairs:
        total = 0
        for nodes in pairs[pair]:
            total += pairs[pair][nodes]
        SummedValues[pair] = total
    return SummedValues
    # returns dictionary with 1849 entries (each possible species pair) with summed scores per species pair

    # NSP = NormaliseSummedPairs(SP)

In [45]:
def NormaliseSummedPairs(SummedPairs):
    SelfSet = set({})
    for pair in SummedPairs:
        if pair[0] == pair[1]:
            SelfSet.add(pair)
        else:        
            A = (pair[0],pair[0])
            B = (pair[1],pair[1])
            MaxVal = max(SummedPairs[A],SummedPairs[B])
            SummedPairs[pair] /= MaxVal
    for pair in SelfSet:
        SummedPairs[pair] = 1.0
        # this function doesn't return anythin, normalizes summed intenisities with max of summed A and B

        # FCN = FindClosestNeighbor(SP,)

In [46]:
def MakeDistanceMatrix(SummedPairs,SampleNames,OutFile,ScalingFactor = -1.0, NameMap=None):
    """
    Takes summed pairs, a matrix of similarity scores (higher = more similar)
    
    Makes an s x s distance matrix comparing all samples to each other
    the output can be passed to phylip to use in tree generation
    
    A 5 x 5 distance matrix to use in phylip looks like this:
    
        5
    Rabbit      0.0000  0.2818  0.9386  0.9830  1.2617
    Human       0.2818  0.0000  1.0795  1.1496  1.5312
    Opossum     0.9386  1.0795  0.0000  1.5380  1.8615
    Chicken     0.9830  1.1496  1.5380  0.0000  1.5819
    Frog        1.2617  1.5312  1.8615  1.5819  0.0000
    """
    #make a len(names)**2 zeros matrix
    #populate according to name order by adding at appropriate index
    #print to file by adding name to each line
    matrix = np.zeros((len(SampleNames),len(SampleNames)))
    for i in range(len(SampleNames)):
        for j in range(len(SampleNames)):
            matrix[i,j] += SummedPairs[(SampleNames[i],SampleNames[j])]
    
    matrix -= 1
    matrix *= ScalingFactor
    
    """
    if NameMap:
        labels = [NameMap[SampleNames[i]] for i in SampleNames]
    else:
        labels = SampleNames
    with open(OutFile,'w') as F:
    """   
    
    #file output not implemented yet.
    return matrix
   
def RunAll(edge_file, node_file,Dendro_name,name_map_file,blank_name):
    
    print("reading node data...")
    nodeDat = ReadNodesFile(node_file,blank_name=blank_name)
    
    print("reading edge data...")
    edgeDat = ReadEdgesFile(edge_file)  
          
    ValidNodes = edgeDat[1]
    CosineScores = edgeDat[0]
    
    print("making sparse matrix...")
    SampleMatrix = MakeSparseMatrix(nodeDat[0],nodeDat[1])
    
    SampleVectors = SampleMatrix[0]
    
    print("Constructing ion abundance similarity matrix: " )
    pairs = MakePairScores(SampleVectors,ValidNodes)
    
    print("weightinng by cosine similarity...")
    WeightByCosine(pairs,CosineScores,SelfBonus = 1,OtherPenalty = 1)
    
    print("Summing similarity matirices...")
    SummedPairs = SumPairs(pairs)
    
    print("Normalising similarity matirices...") 
    NormaliseSummedPairs(SummedPairs)
    
    print("parsing name file...") 
    names = list(nodeDat[0].keys())
    blank_name = None
    name_map_file = None
    NamesNoBlank = [i  for i in names if i != blank_name]#add capability for multiple blank names...
    
    if name_map_file:
        with open(name_map_file,'rb') as F:
            name_map = pickle.load(F)
        NamesConverted = [name_map[i] if i in name_map else i for i in NamesNoBlank]
    
    else:
        NamesConverted = NamesNoBlank
        
    NamesConverted = [SubDict[name] for name in NamesConverted] 
        
    print("making dendrogram..." )    
    mat = MakeDistanceMatrix(SummedPairs,NamesNoBlank,"file")  
    mat = (np.round(mat,decimals = 4))**5
    
    # save distance matrix with names
    df = pd.DataFrame(mat, index=NamesNoBlank, columns=NamesNoBlank)
    # save distance matrix with names
    df = df.replace(-0.0,0)
    df.columns = df.columns.str.replace('_','.')
    df.index = df.index.str.replace('_','.')
    df.to_csv('NamedDistanceMatrix.txt', index=True, header=True, sep='\t')
    # save distance matrix with names
    dists = squareform(mat)
    linkage_matrix = linkage(dists, linkage_type)
    dendrogram(linkage_matrix, labels=NamesConverted,leaf_font_size=2)
    plt.title("test")
    plt.savefig(Dendro_name,bbox_inches = 'tight')  
    plt.close()   
    print(mat)
    np.savetxt("DitanceMatrix.txt", mat)

In [51]:
def main(edge_file,node_file,Dendro_name):
    
    
        edge_file = edge_file
        node_file = node_file
        Dendro_name = Dendro_name
    

        blank_name = None

        name_map_file = None  
        
        RunAll(edge_file, node_file,Dendro_name,name_map_file,blank_name)

Specify the name of your edges table, feature table and a name for the dendrogram output. 

In [52]:
main("AlnusEdges.txt","Alnus_quant.csv","Dendro_Alnus.pdf")

reading node data...
mod1
analysing the following samples:
***
Af-T
Af-L
Af-F
Af-B
Ah-L
Ahv-B
Ah-T
Ah-F
Ahv-F
Ahv-T
Ahv-L
Aj-B
Aj-L
Aj-T
Aj-F
***
reading edge data...
making sparse matrix...
Constructing ion abundance similarity matrix: 
...Making comparisons for sample Af-T
...Making comparisons for sample Af-L
...Making comparisons for sample Af-F
...Making comparisons for sample Af-B
...Making comparisons for sample Ah-L
...Making comparisons for sample Ahv-B
...Making comparisons for sample Ah-T
...Making comparisons for sample Ah-F
...Making comparisons for sample Ahv-F
...Making comparisons for sample Ahv-T
...Making comparisons for sample Ahv-L
...Making comparisons for sample Aj-B
...Making comparisons for sample Aj-L
...Making comparisons for sample Aj-T
...Making comparisons for sample Aj-F
weightinng by cosine similarity...
Summing similarity matirices...
Normalising similarity matirices...
parsing name file...
making dendrogram...
[[-0.          0.0218574   0.0261509   0.00